In [2]:
import tweepy
import time
import sqlite3
from contextlib import closing
import time
from tweepy import TweepError
import os
from dotenv import load_dotenv
from mysql.connector import connect, Error
load_dotenv()

True

In [3]:
consumer_key= os.getenv('consumer_key')
consumer_secret = os.getenv('consumer_secret')
access_token = os.getenv('access_token')
access_token_secret = os.getenv('access_token_secret')

In [4]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [5]:
id_lst =[]
single_call_count = 0
single_call_limit = 5
total_limit_count = 0
total_limit_requests = 899
count = 1

In [6]:
def packaging_data_for_update(hydrated_tweets):
    list_to_update = []
    good_set = set()
    for tweet in hydrated_tweets:
        try:
            text = tweet.retweeted_status.full_text
        except AttributeError:
            text = tweet.full_text
        Created_at = tweet.created_at
        location = tweet.user.location
        followers_count = int(tweet.user.followers_count)
        tweet_id = int(tweet.id)
        good_set.add(int(tweet_id))
        User_id = int(tweet.user.id)
        try:
            RT = tweet.retweeted_status.id
        except AttributeError:
            RT = 'False'
        try:
            retweet_count = tweet.retweeted_status.retweet_count
        except AttributeError:
            retweet_count = tweet.retweet_count
        list_to_update.append([Created_at, location, followers_count, RT, User_id, text, retweet_count, tweet_id])
    return list_to_update, good_set

In [6]:
db='BigData'
with closing(connect(host="localhost", user=os.getenv('USERNAME1'), password=os.getenv('PASSWORD1'), database=db)) as cnxn:
    with closing(cnxn.cursor()) as cursorIter:
        # cursorIter.execute('Select Tweet_ID from Sentiment where User_id is not null ')
        cursorIter.execute('Select Tweet_ID from Sentiment where Fetched is null')
        for row in cursorIter:
            if single_call_count >= single_call_limit:
                print(f'Fetching : {count} calls made')
                try:
                    hydrated_tweets = api.statuses_lookup(id_lst, tweet_mode = "extended")
                except TweepError:
                    time.sleep(60)
                    hydrated_tweets = api.statuses_lookup(id_lst, tweet_mode = "extended")
                packaged_info, good_set = packaging_data_for_update(hydrated_tweets)
                with closing(connect(host="localhost", user=os.getenv('USERNAME1'), password=os.getenv('PASSWORD1'), database=db)) as cnxn1:
                    with closing(cnxn1.cursor(buffered=True)) as cursorUpdater:
                        try:
                            cursorUpdater.executemany('update Sentiment set Created_at = %s, location = %s, followers_count = %s, Original_tweet = %s, User_id = %s, Content = %s, Retweet_count = %s, Fetched = "Yes" where Tweet_ID = %s', packaged_info)
                            cursorUpdater.executemany('update Sentiment set Fetched = "Yes" where Tweet_ID = %s', [[tweet] for tweet in set(id_lst) - good_set])
                        except:
                            pass
                        cnxn1.commit()
                single_call_count = 1
                id_lst = [row[0]]
                count += 1
            else:
                single_call_count += 1
                id_lst.append(row[0])


Rate limit reached. Sleeping for: 54
Fetching : 1 calls made
Fetching : 2 calls made
Fetching : 3 calls made
Fetching : 4 calls made
Fetching : 5 calls made
Fetching : 6 calls made
Fetching : 7 calls made
Fetching : 8 calls made
Fetching : 9 calls made
Fetching : 10 calls made
Fetching : 11 calls made
Fetching : 12 calls made
Fetching : 13 calls made
Fetching : 14 calls made
Fetching : 15 calls made
Fetching : 16 calls made
Fetching : 17 calls made
Fetching : 18 calls made
Fetching : 19 calls made
Fetching : 20 calls made
Fetching : 21 calls made
Fetching : 22 calls made
Fetching : 23 calls made
Fetching : 24 calls made
Fetching : 25 calls made
Fetching : 26 calls made
Fetching : 27 calls made
Fetching : 28 calls made
Fetching : 29 calls made
Fetching : 30 calls made
Fetching : 31 calls made
Fetching : 32 calls made
Fetching : 33 calls made
Fetching : 34 calls made
Fetching : 35 calls made
Fetching : 36 calls made
Fetching : 37 calls made
Fetching : 38 calls made
Fetching : 39 calls ma

InternalError: Unread result found

In [7]:
lst1 = set([1, 2 ,3])
lst2 = set([2, 3, 4])
print([x for x in lst2])

[2, 3, 4]


In [13]:
example = api.statuses_lookup([1289350148263251969, 1289350148334657537], tweet_mode="extended")


In [54]:
# print(example[0].retweeted_status.full_text)
try:
    print(example[1].retweeted_status.id)
except AttributeError:
    print(example[0].retweet_count)
# followers_count = int(tweet._json['user']['followers_count'])
# User_id = int(tweet._json['user']['id'])
# RT = bool(re.match('^RT', tweet._json['text']))
# tweet_id = int(tweet._json['id'])
# text = tweet._json['text']
print(example[0].retweeted_status)

1289309756415336448


AttributeError: 'Status' object has no attribute 'retweeted_status'

In [38]:
import json
with open('test.txt', 'w') as f:
    json.dump(example[0]._json, f)
    json.dump(example[1]._json, f)

In [10]:
import json
import re
status = example[0]
print(status._json['created_at'])
print(status._json['user']['location'])
print(status._json['user']['followers_count'])
print(status._json['user']['id'])
print(re.match('^RT', status._json['text']))
print(status._json['id'])
# text
# created_at
# location
# followers_count

IndexError: list index out of range